In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession \
        .builder \
        .appName("Test") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()

df=spark.read.csv("/content/Connecticut_Healthcare_Facilities_20240313.csv",header=True,inferSchema=True)
print(df.dtypes)
df.printSchema()

[('Credential ID', 'int'), ('Name', 'string'), ('Type', 'string'), ('Business Name', 'string'), ('DBA', 'string'), ('Full Credential Code', 'string'), ('Credential Type', 'string'), ('Credential Number', 'string'), ('Credential Subcategory', 'string'), ('Credential', 'string'), ('Status', 'string'), ('Status Reason', 'string'), ('Active', 'int'), ('Issue Date', 'string'), ('Effective Date', 'string'), ('Expiration Date', 'string'), ('Address', 'string'), ('City', 'string'), ('State', 'string'), ('Zip Code', 'int'), ('Date Record Refreshed ', 'string'), ('Location', 'string')]
root
 |-- Credential ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Business Name: string (nullable = true)
 |-- DBA: string (nullable = true)
 |-- Full Credential Code: string (nullable = true)
 |-- Credential Type: string (nullable = true)
 |-- Credential Number: string (nullable = true)
 |-- Credential Subcategory: string (nullable = true)
 |-- Credent

In [ ]:
print(df.count())
print(df.distinct().count())

1000
1000


In [ ]:
df.select('City').distinct().show(df.count())
df.select('City').distinct().count()


+----------------+
|            City|
+----------------+
|       WATERTOWN|
|       ELLINGTON|
|        STAMFORD|
|        DAYVILLE|
|      MIDDLEBURY|
|      PLAINVILLE|
|      Litchfield|
|      RIDGEFIELD|
|            AVON|
|        Trumball|
|     NEW PRESTON|
|         CHESTER|
|      Woodbridge|
|       WATERBURY|
|       DANIELSON|
|        Guilford|
|         TOLLAND|
|      Manchester|
|          Wilton|
|      UNCASVILLE|
|         ANSONIA|
|        PORTLAND|
|      FARMINGTON|
|         WINDHAM|
|        PROSPECT|
|          Orange|
|       Newington|
|        SIMSBURY|
|       GREENWICH|
|         NIANTIC|
|      LITCHFIELD|
|       Fairfield|
|          DARIEN|
|          MYSTIC|
|       WESTBROOK|
|     WALLINGFORD|
|      DEEP RIVER|
|         ENFIELD|
|      Farmington|
|         Norwich|
|          Darien|
|      EAST HAVEN|
|         MADISON|
|     GLASTONBURY|
|        CHESHIRE|
|        WESTPORT|
|       NEWINGTON|
|     Southington|
|        GUILFORD|
|    COLLINS

168

In [ ]:
for col in df.columns:
    null_count = df.filter(df[col].isNull() | (df[col] == "")).count()
    print(f"Column: {col}\t Null Count: {null_count}")
total_count = df.count()
print(f"Total Count: {total_count}")

Column: Credential ID	 Null Count: 0
Column: Name	 Null Count: 0
Column: Type	 Null Count: 0
Column: Business Name	 Null Count: 1
Column: DBA	 Null Count: 2
Column: Full Credential Code	 Null Count: 0
Column: Credential Type	 Null Count: 0
Column: Credential Number	 Null Count: 26
Column: Credential Subcategory	 Null Count: 1000
Column: Credential	 Null Count: 0
Column: Status	 Null Count: 0
Column: Status Reason	 Null Count: 97
Column: Active	 Null Count: 0
Column: Issue Date	 Null Count: 48
Column: Effective Date	 Null Count: 45
Column: Expiration Date	 Null Count: 34
Column: Address	 Null Count: 1
Column: City	 Null Count: 0
Column: State	 Null Count: 0
Column: Zip Code	 Null Count: 0
Column: Date Record Refreshed 	 Null Count: 0
Column: Location	 Null Count: 0
Total Count: 1000


In [ ]:
desired_types = ["ACTIVE", "ACTIVE IN RENEWAL"]
# Filter the DataFrame based on the list
df = df.filter(df["Status"].isin(desired_types))
print(df.count())

529


In [ ]:
for col in df.columns:
    null_count = df.filter(df[col].isNull() | (df[col] == "")).count()
    print(f"Column: {col}\t Null Count: {null_count}")
total_count = df.count()
print(f"Total Count: {total_count}")

Column: Credential ID	 Null Count: 0
Column: Name	 Null Count: 0
Column: Type	 Null Count: 0
Column: Business Name	 Null Count: 0
Column: DBA	 Null Count: 1
Column: Full Credential Code	 Null Count: 0
Column: Credential Type	 Null Count: 0
Column: Credential Number	 Null Count: 0
Column: Credential Subcategory	 Null Count: 529
Column: Credential	 Null Count: 0
Column: Status	 Null Count: 0
Column: Status Reason	 Null Count: 0
Column: Active	 Null Count: 0
Column: Issue Date	 Null Count: 2
Column: Effective Date	 Null Count: 0
Column: Expiration Date	 Null Count: 0
Column: Address	 Null Count: 0
Column: City	 Null Count: 0
Column: State	 Null Count: 0
Column: Zip Code	 Null Count: 0
Column: Date Record Refreshed 	 Null Count: 0
Column: Location	 Null Count: 0
Total Count: 529


In [ ]:
df = df.drop("Credential Subcategory", "Active", "Status Reason", "State", "Date Record Refreshed ", "Credential Type", "Credential Number")
df.printSchema()

root
 |-- Credential ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Business Name: string (nullable = true)
 |-- DBA: string (nullable = true)
 |-- Full Credential Code: string (nullable = true)
 |-- Credential: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Effective Date: string (nullable = true)
 |-- Expiration Date: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zip Code: integer (nullable = true)
 |-- Location: string (nullable = true)



In [ ]:
# Replace null values in DBA column with values from Business Name column
df = df.na.fill({'DBA': 'Business Name'})

# Replace null values in Issue Date column with values from Effective Date column
df = df.na.fill({'Issue Date': 'Effective Date'})



In [ ]:
for col in df.columns:
    null_count = df.filter(df[col].isNull() | (df[col] == "")).count()
    print(f"Column: {col}\t Null Count: {null_count}")
total_count = df.count()
print(f"Total Count: {total_count}")

Column: Credential ID	 Null Count: 0
Column: Name	 Null Count: 0
Column: Type	 Null Count: 0
Column: Business Name	 Null Count: 0
Column: DBA	 Null Count: 0
Column: Full Credential Code	 Null Count: 0
Column: Credential	 Null Count: 0
Column: Status	 Null Count: 0
Column: Issue Date	 Null Count: 0
Column: Effective Date	 Null Count: 0
Column: Expiration Date	 Null Count: 0
Column: Address	 Null Count: 0
Column: City	 Null Count: 0
Column: Zip Code	 Null Count: 0
Column: Location	 Null Count: 0
Total Count: 529


In [ ]:
from pyspark.sql.functions import regexp_replace
df = df.withColumn("Issue Date", regexp_replace("Issue Date", "\s+\d{2}:\d{2}:\d{2}\s+(AM|PM)?", ""))
df = df.withColumn("Issue Date", regexp_replace("Issue Date", " 0:00", ""))
df.select('Issue Date').show(df.count())




+--------------+
|    Issue Date|
+--------------+
|    06/22/2011|
|    04/01/2010|
|    06/30/2017|
|    12/14/2006|
|    12/31/1999|
|    10/01/2009|
|    04/01/2008|
|    09/01/2022|
|    06/22/2004|
|    10/24/2019|
|    12/09/2015|
|    03/06/2020|
|    01/01/2008|
|    01/14/2019|
|    07/01/2009|
|    01/01/2010|
|    12/18/2014|
|    10/11/2018|
|    05/22/2006|
|    06/20/2016|
|    09/23/2003|
|    12/05/2016|
|    12/01/2008|
|    10/01/2009|
|    11/07/2021|
|    04/01/2010|
|    11/09/2016|
|    08/01/2019|
|    01/01/2010|
|    07/01/2009|
|    06/20/2016|
|    11/29/2023|
|    12/01/2008|
|    10/07/2020|
|    07/22/2022|
|    10/30/2008|
|    06/08/2004|
|    07/02/2021|
|    01/11/1993|
|    07/28/2006|
|    06/24/2022|
|    06/04/2010|
|    06/22/2011|
|    04/01/2015|
|    12/13/2005|
|    01/05/2023|
|    06/23/2011|
|    09/25/2023|
|    10/01/2008|
|    10/01/2008|
|    09/11/2023|
|    10/10/2007|
|    10/07/2019|
|    10/01/2009|
|    07/01/2008|
|    10/01/201

In [ ]:
from pyspark.sql.functions import to_date, date_format
df = df.withColumn("Issue Date", to_date(df["Issue Date"], "M/d/yyyy"))
df = df.withColumn("Issue Date", date_format("Issue Date", "MM/dd/yyyy"))
df.select('Issue Date').show(df.count())
df.select('Issue Date').printSchema()





+----------+
|Issue Date|
+----------+
|06/22/2011|
|04/01/2010|
|06/30/2017|
|12/14/2006|
|12/31/1999|
|10/01/2009|
|04/01/2008|
|09/01/2022|
|06/22/2004|
|10/24/2019|
|12/09/2015|
|03/06/2020|
|01/01/2008|
|01/14/2019|
|07/01/2009|
|01/01/2010|
|12/18/2014|
|10/11/2018|
|05/22/2006|
|06/20/2016|
|09/23/2003|
|12/05/2016|
|12/01/2008|
|10/01/2009|
|11/07/2021|
|04/01/2010|
|11/09/2016|
|08/01/2019|
|01/01/2010|
|07/01/2009|
|06/20/2016|
|11/29/2023|
|12/01/2008|
|10/07/2020|
|07/22/2022|
|10/30/2008|
|06/08/2004|
|07/02/2021|
|01/11/1993|
|07/28/2006|
|06/24/2022|
|06/04/2010|
|06/22/2011|
|04/01/2015|
|12/13/2005|
|01/05/2023|
|06/23/2011|
|09/25/2023|
|10/01/2008|
|10/01/2008|
|09/11/2023|
|10/10/2007|
|10/07/2019|
|10/01/2009|
|07/01/2008|
|10/01/2015|
|09/14/2022|
|09/20/2018|
|01/01/2010|
|07/19/2001|
|02/08/2017|
|12/02/2008|
|10/01/2008|
|07/01/2008|
|03/18/2005|
|05/23/2017|
|10/01/2008|
|09/28/2012|
|09/14/2022|
|01/04/2024|
|07/01/2008|
|09/24/2020|
|10/31/2016|
|08/29/2023|

In [ ]:
df = df.withColumn("Effective Date", regexp_replace("Effective Date", "\s+\d{2}:\d{2}:\d{2}\s+(AM|PM)?", ""))
df = df.withColumn("Effective Date", regexp_replace("Effective Date", " 0:00", ""))
df.select('Effective Date').show(df.count())

+--------------+
|Effective Date|
+--------------+
|    04/01/2023|
|    04/01/2022|
|    04/01/2023|
|    04/01/2023|
|    04/01/2022|
|    10/01/2023|
|    04/01/2022|
|    09/01/2022|
|    07/01/2022|
|    10/01/2023|
|    10/01/2023|
|    04/01/2022|
|    01/01/2022|
|    10/01/2022|
|    07/01/2021|
|    01/01/2022|
|    10/01/2022|
|    10/01/2022|
|    07/01/2022|
|    04/01/2022|
|    10/01/2023|
|    10/01/2022|
|    10/01/2022|
|    10/01/2023|
|    10/01/2023|
|    04/01/2022|
|    10/01/2022|
|    07/01/2023|
|    01/01/2024|
|    07/01/2023|
|    04/01/2022|
|    11/28/2023|
|    10/01/2022|
|    10/01/2022|
|    07/17/2022|
|    10/01/2022|
|    07/01/2022|
|    07/01/2023|
|    04/01/2023|
|    01/01/2024|
|    06/23/2022|
|    04/01/2022|
|    04/01/2023|
|    04/01/2023|
|    01/01/2024|
|    01/05/2023|
|    04/01/2023|
|    09/26/2023|
|    10/01/2022|
|    10/01/2022|
|    09/11/2023|
|    04/01/2022|
|    10/01/2023|
|    10/01/2023|
|    07/01/2022|
|    10/01/202

In [ ]:
from pyspark.sql.functions import to_date, date_format
df = df.withColumn("Effective Date", to_date(df["Effective Date"], "M/d/yyyy"))
df = df.withColumn("Effective Date", date_format("Effective Date", "MM/dd/yyyy"))
df.select('Effective Date').show(df.count())
df.select('Effective Date').printSchema()

+--------------+
|Effective Date|
+--------------+
|    04/01/2023|
|    04/01/2022|
|    04/01/2023|
|    04/01/2023|
|    04/01/2022|
|    10/01/2023|
|    04/01/2022|
|    09/01/2022|
|    07/01/2022|
|    10/01/2023|
|    10/01/2023|
|    04/01/2022|
|    01/01/2022|
|    10/01/2022|
|    07/01/2021|
|    01/01/2022|
|    10/01/2022|
|    10/01/2022|
|    07/01/2022|
|    04/01/2022|
|    10/01/2023|
|    10/01/2022|
|    10/01/2022|
|    10/01/2023|
|    10/01/2023|
|    04/01/2022|
|    10/01/2022|
|    07/01/2023|
|    01/01/2024|
|    07/01/2023|
|    04/01/2022|
|    11/28/2023|
|    10/01/2022|
|    10/01/2022|
|    07/17/2022|
|    10/01/2022|
|    07/01/2022|
|    07/01/2023|
|    04/01/2023|
|    01/01/2024|
|    06/23/2022|
|    04/01/2022|
|    04/01/2023|
|    04/01/2023|
|    01/01/2024|
|    01/05/2023|
|    04/01/2023|
|    09/26/2023|
|    10/01/2022|
|    10/01/2022|
|    09/11/2023|
|    04/01/2022|
|    10/01/2023|
|    10/01/2023|
|    07/01/2022|
|    10/01/202

In [ ]:
df = df.withColumn("Expiration Date", regexp_replace("Expiration Date", "\s+\d{2}:\d{2}:\d{2}\s+(AM|PM)?", ""))
df = df.withColumn("Expiration Date", regexp_replace("Expiration Date", " 0:00", ""))
df.select('Expiration Date').show(df.count())

+---------------+
|Expiration Date|
+---------------+
|     03/31/2025|
|     03/31/2024|
|     03/31/2025|
|     03/31/2025|
|     03/31/2024|
|     09/30/2025|
|     03/31/2024|
|     06/30/2024|
|     06/30/2024|
|     09/30/2025|
|     09/30/2025|
|     06/30/2024|
|     12/31/2023|
|     09/30/2024|
|     06/30/2023|
|     12/31/2023|
|     09/30/2024|
|     09/30/2024|
|     06/30/2024|
|     03/31/2024|
|     09/30/2025|
|     09/30/2024|
|     09/30/2024|
|     09/30/2025|
|     09/30/2025|
|     03/31/2024|
|     09/30/2024|
|     06/30/2025|
|     12/31/2025|
|     06/30/2025|
|     03/31/2024|
|     09/30/2025|
|     09/30/2024|
|     09/30/2024|
|     06/30/2024|
|     09/30/2024|
|     06/30/2024|
|     06/30/2025|
|     03/31/2025|
|     12/31/2025|
|     03/31/2024|
|     03/31/2024|
|     03/31/2025|
|     03/31/2025|
|     12/31/2025|
|     12/31/2024|
|     03/31/2025|
|     09/30/2025|
|     09/30/2024|
|     09/30/2024|
|     09/30/2025|
|     03/31/2024|
|     09/3

In [ ]:
from pyspark.sql.functions import to_date, date_format
df = df.withColumn("Expiration Date", to_date(df["Expiration Date"], "M/d/yyyy"))
df = df.withColumn("Expiration Date", date_format("Expiration Date", "MM/dd/yyyy"))
df.select('Expiration Date').show(df.count())
df.select('Expiration Date').printSchema()

+---------------+
|Expiration Date|
+---------------+
|     03/31/2025|
|     03/31/2024|
|     03/31/2025|
|     03/31/2025|
|     03/31/2024|
|     09/30/2025|
|     03/31/2024|
|     06/30/2024|
|     06/30/2024|
|     09/30/2025|
|     09/30/2025|
|     06/30/2024|
|     12/31/2023|
|     09/30/2024|
|     06/30/2023|
|     12/31/2023|
|     09/30/2024|
|     09/30/2024|
|     06/30/2024|
|     03/31/2024|
|     09/30/2025|
|     09/30/2024|
|     09/30/2024|
|     09/30/2025|
|     09/30/2025|
|     03/31/2024|
|     09/30/2024|
|     06/30/2025|
|     12/31/2025|
|     06/30/2025|
|     03/31/2024|
|     09/30/2025|
|     09/30/2024|
|     09/30/2024|
|     06/30/2024|
|     09/30/2024|
|     06/30/2024|
|     06/30/2025|
|     03/31/2025|
|     12/31/2025|
|     03/31/2024|
|     03/31/2024|
|     03/31/2025|
|     03/31/2025|
|     12/31/2025|
|     12/31/2024|
|     03/31/2025|
|     09/30/2025|
|     09/30/2024|
|     09/30/2024|
|     09/30/2025|
|     03/31/2024|
|     09/3

In [ ]:
from pyspark.sql.functions import col
# Convert "Zip Code" column to integer
df = df.withColumn("Zip Code", col("Zip Code").cast("int"))

# Define UDF to process zip codes
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

def process_zip(zip_code):
    zip_str = str(zip_code)
    if len(zip_str) > 4:
        return int(zip_str[:-4])
    else:
        return zip_code

# Register UDF
process_zip_udf = udf(process_zip, IntegerType())

# Apply UDF to "Zip Code" column
df = df.withColumn("Zip Code", process_zip_udf("Zip Code"))

df.select('Zip Code').show(df.count())



+--------+
|Zip Code|
+--------+
|    6517|
|    6611|
|    6385|
|    6355|
|    6706|
|    6106|
|    6280|
|    6355|
|    6477|
|    6831|
|    6010|
|    6477|
|    6479|
|    6426|
|    6033|
|    6268|
|    6239|
|    6074|
|    6451|
|    6488|
|    6830|
|    6471|
|    6512|
|    6450|
|    6890|
|    6840|
|    6492|
|    6067|
|    6790|
|    6469|
|    6488|
|    6525|
|    6877|
|    6234|
|    6226|
|    6095|
|    6492|
|    6084|
|    6790|
|    6484|
|    6877|
|    6457|
|    6385|
|    6410|
|    6510|
|    6334|
|    6905|
|    6226|
|    6062|
|    6896|
|    6824|
|    6902|
|    6606|
|    6831|
|    6488|
|    6040|
|    6795|
|    6437|
|    6702|
|    6437|
|    6902|
|    6451|
|    6896|
|    6082|
|    6320|
|    6606|
|    6117|
|    6477|
|    6033|
|    6032|
|    6385|
|    6033|
|    6320|
|    6850|
|    6810|
|    6106|
|    6705|
|    6320|
|    6360|
|    6355|
|    6001|
|    6076|
|    6484|
|    6037|
|    6513|
|    6457|
|    6360|
|    6484|

In [ ]:
from pyspark.sql.functions import col, concat, lit
df= df.withColumn("Zip Code", concat(lit("0"), col("Zip Code")))
df.select('Zip Code').show()
df.select('Zip Code').printSchema()

+--------+
|Zip Code|
+--------+
|   06517|
|   06611|
|   06385|
|   06355|
|   06706|
|   06106|
|   06280|
|   06355|
|   06477|
|   06831|
|   06010|
|   06477|
|   06479|
|   06426|
|   06033|
|   06268|
|   06239|
|   06074|
|   06451|
|   06488|
+--------+
only showing top 20 rows

root
 |-- Zip Code: string (nullable = true)



In [ ]:
df.select('Zip Code').show()

+--------+
|Zip Code|
+--------+
|   06517|
|   06611|
|   06385|
|   06355|
|   06706|
|   06106|
|   06280|
|   06355|
|   06477|
|   06831|
|   06010|
|   06477|
|   06479|
|   06426|
|   06033|
|   06268|
|   06239|
|   06074|
|   06451|
|   06488|
+--------+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import monotonically_increasing_id
df = df.withColumn("Healthcare ID", monotonically_increasing_id())
df.show()

+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+----------+--------------+---------------+--------------------+-------------+--------+--------------------+-------------+
|Credential ID|                Name|                Type|       Business Name|                 DBA|Full Credential Code|          Credential|           Status|Issue Date|Effective Date|Expiration Date|             Address|         City|Zip Code|            Location|Healthcare ID|
+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+----------+--------------+---------------+--------------------+-------------+--------+--------------------+-------------+
|      1003908|  ATRIA LARSON PLACE|LIMITED LIABILITY...|  ATRIA LARSON PLACE|  ATRIA LARSON PLACE|        ALSA.0000161|Assisted Living S...|           ACTIV

In [ ]:
# Write the DataFrame to a new CSV file
df.write.csv("Health_Updated.csv",header=True)

# Download the CSV file to the system
from google.colab import files
files.download("Health_Updated.csv")